In [1]:
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

# lakes_gdf = gpd.read_file('Data/NH_Wildlife_Action_Plan_2020_-_Aquatic_Habitat_(Lakes_Ponds).geojson')
wetlands_gdf = gpd.read_file('Data/NH_Wetlands_Geopackage.gpkg', layer='NH_Wetlands')
bloom_data = gpd.read_file("Data/cyanobacteria_geocoded_full5.gpkg")

wetlands_gdf = wetlands_gdf.to_crs("EPSG:3857")
bloom_data = bloom_data.to_crs("EPSG:3857")

In [2]:
lakes = wetlands_gdf[wetlands_gdf['WETLAND_TYPE'].isin(["Lake", "Freshwater Pond"])]
wetlands = wetlands_gdf[wetlands_gdf['WETLAND_TYPE'].isin(["Estuarine and Marine Wetland",
                                                          "Freshwater Emergent Wetland",
                                                          "Freshwater Forested/Shrub Wetland"])]

merged_blooms_lakes = gpd.sjoin(bloom_data, lakes, how='inner', op='intersects')

merged_blooms_lakes.drop(columns=['identifier', 'date_advisory_removed', 'auid', 'Cleaned_Name', 'gnis_name', 'statecode', 'fdate', 'resolution', 'reachcode', 'ftype', 'fcode', 'visibilityfilter', 'nhdplusid', 'vpuid', 'GlobalID', 'index_right'], inplace=True)

In [12]:
# Group by 'lake' and 'date_advisory_issued' and aggregate the data
aggregated_bloom_data = merged_blooms_lakes.groupby(['lake', 'date_advisory_issued'], as_index=False).agg({  # Count the number of blooms for each combination of lake and advisory date
    'geometry': 'first',  # Keep the geometry of the first bloom for each group (or adjust if needed)
    'lake': 'first',  # Add more columns here and choose appropriate aggregation functions for each
    'town': 'first', 'dominant_taxa': 'first', 'total_cell_concentration': 'first',
    'advisory_days': 'first', 'notification_type': 'first', 'year': 'first', 'YearMonth': 'first',
    'Shape_Length': 'first', 'Shape_Area': 'first', 'permanent_identifier': 'first', 'gnis_id': 'first',
    'areasqkm': 'first', 'elevation': 'first', 'ATTRIBUTE': 'unique',
    'WETLAND_TYPE': 'unique', 'ACRES': 'sum', 'NWI_ID': 'first'
    # Example:
    # 'SomeOtherColumn': 'sum',  # If you want to sum a numeric column
    # 'AnotherColumn': 'unique',  # If you want to list unique values in a column
})

# Check the result
aggregated_bloom_data = aggregated_bloom_data.set_geometry('geometry')
aggregated_bloom_data

,date_advisory_issued,geometry,lake,town,dominant_taxa,total_cell_concentration,advisory_days,notification_type,year,YearMonth,Shape_Length,Shape_Area,permanent_identifier,gnis_id,areasqkm,elevation,ATTRIBUTE,WETLAND_TYPE,ACRES,NWI_ID
0,2019-10-11,"MULTIPOLYGON (((-7993409.975 5380279.400, -799...",Adder Pond/Hopkins Pond,Andover,Anabaena/Dolichospermum,"3,580,000",41.0,Warning,2019,2019-10,0.015388,0.000012,135993959,00865203,0.109221,196.3,"[L1ABH, L1UBH]",[Lake],28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...
1,2022-10-06,"MULTIPOLYGON (((-7993409.975 5380279.400, -799...",Adder Pond/Hopkins Pond,Andover,Chrysosporum,"3,100,000",43.0,Warning,2022,2022-10,0.015388,0.000012,135993959,00865203,0.109221,196.3,"[L1ABH, L1UBH]",[Lake],28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...
2,2010-05-25,"MULTIPOLYGON (((-7921916.903 5296831.789, -792...",Angle Pond,Sandown,Anabaena,"235,000",9.0,Warning,2010,2010-05,0.049024,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...
3,2021-08-25,"MULTIPOLYGON (((-7921916.903 5296831.789, -792...",Angle Pond,Sandown,None,None,NaN,Alert,2021,2021-08,0.049024,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...
4,2022-09-12,"MULTIPOLYGON (((-7921916.903 5296831.789, -792...",Angle Pond,Sandown,Dolichospermum,"138,200",7.0,Warning,2022,2022-09,0.049024,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2020-08-09,"MULTIPOLYGON (((-7967191.988 5415879.976, -796...","Winona, Lake",New Hampton,None,None,NaN,Alert,2020,2020-08,NaN,NaN,None,None,NaN,NaN,[L1UBH],[Lake],109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...
655,2023-07-27,"MULTIPOLYGON (((-7967191.988 5415879.976, -796...","Winona, Lake",New Hampton,Dolichospermum,"950,000",7.0,Warning,2023,2023-07,NaN,NaN,None,None,NaN,NaN,[L1UBH],[Lake],109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...
656,2005-08-03,"MULTIPOLYGON (((-7941459.002 5543510.290, -794...",York Pond,Berlin,Aphanizomenon,">70,000 or >50%",48.0,Warning,2005,2005-08,0.016025,0.000010,91154789,00871018,0.087565,455.1,[L2UBHh],[Lake],23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...
657,2008-07-30,"MULTIPOLYGON (((-7941459.002 5543510.290, -794...",York Pond,Berlin,Aphanizomenon,">70,000 or >50%",124.0,Warning,2008,2008-07,0.016025,0.000010,91154789,00871018,0.087565,455.1,[L2UBHh],[Lake],23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...


In [13]:
aggregated_bloom_data['buffer'] = aggregated_bloom_data.geometry.buffer(1000)

In [14]:
aggregated_bloom_data

,date_advisory_issued,geometry,lake,town,dominant_taxa,total_cell_concentration,advisory_days,notification_type,year,YearMonth,...,Shape_Area,permanent_identifier,gnis_id,areasqkm,elevation,ATTRIBUTE,WETLAND_TYPE,ACRES,NWI_ID,buffer
0,2019-10-11,"MULTIPOLYGON (((-7993409.975 5380279.400, -799...",Adder Pond/Hopkins Pond,Andover,Anabaena/Dolichospermum,"3,580,000",41.0,Warning,2019,2019-10,...,0.000012,135993959,00865203,0.109221,196.3,"[L1ABH, L1UBH]",[Lake],28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...,"POLYGON ((-7994994.165 5380215.739, -7994994.1..."
1,2022-10-06,"MULTIPOLYGON (((-7993409.975 5380279.400, -799...",Adder Pond/Hopkins Pond,Andover,Chrysosporum,"3,100,000",43.0,Warning,2022,2022-10,...,0.000012,135993959,00865203,0.109221,196.3,"[L1ABH, L1UBH]",[Lake],28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...,"POLYGON ((-7994994.165 5380215.739, -7994994.1..."
2,2010-05-25,"MULTIPOLYGON (((-7921916.903 5296831.789, -792...",Angle Pond,Sandown,Anabaena,"235,000",9.0,Warning,2010,2010-05,...,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
3,2021-08-25,"MULTIPOLYGON (((-7921916.903 5296831.789, -792...",Angle Pond,Sandown,None,None,NaN,Alert,2021,2021-08,...,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
4,2022-09-12,"MULTIPOLYGON (((-7921916.903 5296831.789, -792...",Angle Pond,Sandown,Dolichospermum,"138,200",7.0,Warning,2022,2022-09,...,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2020-08-09,"MULTIPOLYGON (((-7967191.988 5415879.976, -796...","Winona, Lake",New Hampton,None,None,NaN,Alert,2020,2020-08,...,NaN,None,None,NaN,NaN,[L1UBH],[Lake],109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...,"POLYGON ((-7966193.193 5415929.044, -7966193.1..."
655,2023-07-27,"MULTIPOLYGON (((-7967191.988 5415879.976, -796...","Winona, Lake",New Hampton,Dolichospermum,"950,000",7.0,Warning,2023,2023-07,...,NaN,None,None,NaN,NaN,[L1UBH],[Lake],109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...,"POLYGON ((-7966193.193 5415929.044, -7966193.1..."
656,2005-08-03,"MULTIPOLYGON (((-7941459.002 5543510.290, -794...",York Pond,Berlin,Aphanizomenon,">70,000 or >50%",48.0,Warning,2005,2005-08,...,0.000010,91154789,00871018,0.087565,455.1,[L2UBHh],[Lake],23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...,"POLYGON ((-7942431.517 5543743.130, -7942426.0..."
657,2008-07-30,"MULTIPOLYGON (((-7941459.002 5543510.290, -794...",York Pond,Berlin,Aphanizomenon,">70,000 or >50%",124.0,Warning,2008,2008-07,...,0.000010,91154789,00871018,0.087565,455.1,[L2UBHh],[Lake],23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...,"POLYGON ((-7942431.517 5543743.130, -7942426.0..."


In [15]:
aggregated_bloom_data.geometry

0      MULTIPOLYGON (((-7993409.975 5380279.400, -799...
1      MULTIPOLYGON (((-7993409.975 5380279.400, -799...
2      MULTIPOLYGON (((-7921916.903 5296831.789, -792...
3      MULTIPOLYGON (((-7921916.903 5296831.789, -792...
4      MULTIPOLYGON (((-7921916.903 5296831.789, -792...
                             ...                        
654    MULTIPOLYGON (((-7967191.988 5415879.976, -796...
655    MULTIPOLYGON (((-7967191.988 5415879.976, -796...
656    MULTIPOLYGON (((-7941459.002 5543510.290, -794...
657    MULTIPOLYGON (((-7941459.002 5543510.290, -794...
658    MULTIPOLYGON (((-7941459.002 5543510.290, -794...
Name: geometry, Length: 659, dtype: geometry

In [33]:
# aggregated_bloom_data.drop('geometry', axis=1, inplace=True)
# aggregated_bloom_data.set_geometry('buffer', inplace=True)

aggregated_bloom_data.to_file('aggregated_bloom_data_buffer_1km.gpkg', driver='GPKG')

ValueError: Invalid field type <class 'numpy.ndarray'>

In [17]:
aggregated_bloom_data.dtypes

date_advisory_issued        datetime64[ns]
geometry                          geometry
lake                                object
town                                object
dominant_taxa                       object
total_cell_concentration            object
advisory_days                      float64
notification_type                   object
year                                 int64
YearMonth                           object
Shape_Length                       float64
Shape_Area                         float64
permanent_identifier                object
gnis_id                             object
areasqkm                           float64
elevation                          float64
ATTRIBUTE                           object
WETLAND_TYPE                        object
ACRES                              float64
NWI_ID                              object
buffer                            geometry
dtype: object

In [20]:
geometry_columns = aggregated_bloom_data.select_dtypes(include=['geometry']).columns
geometry_columns

Index(['buffer'], dtype='object')

In [19]:
aggregated_bloom_data['geometry'] = aggregated_bloom_data['geometry'].apply(lambda geom: geom.wkt)

In [25]:
aggregated_bloom_data = aggregated_bloom_data.set_geometry('buffer')

In [32]:
print(aggregated_bloom_data.geometry)

0      POLYGON ((-7994994.165 5380215.739, -7994994.1...
1      POLYGON ((-7994994.165 5380215.739, -7994994.1...
2      POLYGON ((-7923107.436 5297419.249, -7923104.0...
3      POLYGON ((-7923107.436 5297419.249, -7923104.0...
4      POLYGON ((-7923107.436 5297419.249, -7923104.0...
                             ...                        
654    POLYGON ((-7966193.193 5415929.044, -7966193.1...
655    POLYGON ((-7966193.193 5415929.044, -7966193.1...
656    POLYGON ((-7942431.517 5543743.130, -7942426.0...
657    POLYGON ((-7942431.517 5543743.130, -7942426.0...
658    POLYGON ((-7942431.517 5543743.130, -7942426.0...
Name: buffer, Length: 659, dtype: geometry


In [31]:
from shapely.geometry import shape
import numpy as np

# Assuming 'geometry' is your geometry column
aggregated_bloom_data['buffer'] = aggregated_bloom_data['buffer'].apply(lambda x: shape(x) if isinstance(x, np.ndarray) else x)


In [34]:
aggregated_bloom_data

,date_advisory_issued,lake,town,dominant_taxa,total_cell_concentration,advisory_days,notification_type,year,YearMonth,Shape_Length,Shape_Area,permanent_identifier,gnis_id,areasqkm,elevation,ATTRIBUTE,WETLAND_TYPE,ACRES,NWI_ID,buffer
0,2019-10-11,Adder Pond/Hopkins Pond,Andover,Anabaena/Dolichospermum,"3,580,000",41.0,Warning,2019,2019-10,0.015388,0.000012,135993959,00865203,0.109221,196.3,"[L1ABH, L1UBH]",[Lake],28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...,"POLYGON ((-7994994.165 5380215.739, -7994994.1..."
1,2022-10-06,Adder Pond/Hopkins Pond,Andover,Chrysosporum,"3,100,000",43.0,Warning,2022,2022-10,0.015388,0.000012,135993959,00865203,0.109221,196.3,"[L1ABH, L1UBH]",[Lake],28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...,"POLYGON ((-7994994.165 5380215.739, -7994994.1..."
2,2010-05-25,Angle Pond,Sandown,Anabaena,"235,000",9.0,Warning,2010,2010-05,0.049024,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
3,2021-08-25,Angle Pond,Sandown,None,None,NaN,Alert,2021,2021-08,0.049024,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
4,2022-09-12,Angle Pond,Sandown,Dolichospermum,"138,200",7.0,Warning,2022,2022-09,0.049024,0.000072,135909961,00865245,0.650459,67.1,"[L1UBH, L1ABH]",[Lake],161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2020-08-09,"Winona, Lake",New Hampton,None,None,NaN,Alert,2020,2020-08,NaN,NaN,None,None,NaN,NaN,[L1UBH],[Lake],109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...,"POLYGON ((-7966193.193 5415929.044, -7966193.1..."
655,2023-07-27,"Winona, Lake",New Hampton,Dolichospermum,"950,000",7.0,Warning,2023,2023-07,NaN,NaN,None,None,NaN,NaN,[L1UBH],[Lake],109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...,"POLYGON ((-7966193.193 5415929.044, -7966193.1..."
656,2005-08-03,York Pond,Berlin,Aphanizomenon,">70,000 or >50%",48.0,Warning,2005,2005-08,0.016025,0.000010,91154789,00871018,0.087565,455.1,[L2UBHh],[Lake],23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...,"POLYGON ((-7942431.517 5543743.130, -7942426.0..."
657,2008-07-30,York Pond,Berlin,Aphanizomenon,">70,000 or >50%",124.0,Warning,2008,2008-07,0.016025,0.000010,91154789,00871018,0.087565,455.1,[L2UBHh],[Lake],23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...,"POLYGON ((-7942431.517 5543743.130, -7942426.0..."


In [35]:
# Check the data type of the geometry column
print(aggregated_bloom_data['buffer'].dtype)


geometry


In [47]:
# Check the type of the first few entries in the 'buffer' column
print([type(x) for x in aggregated_bloom_data['buffer'].head()])


[<class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>]


In [37]:
aggregated_bloom_data.to_file('aggregated_bloom_data_buffer_1km.gpkg', driver='GPKG')

ValueError: Invalid field type <class 'numpy.ndarray'>

In [38]:
print(aggregated_bloom_data.dtypes)

date_advisory_issued        datetime64[ns]
lake                                object
town                                object
dominant_taxa                       object
total_cell_concentration            object
advisory_days                      float64
notification_type                   object
year                                 int64
YearMonth                           object
Shape_Length                       float64
Shape_Area                         float64
permanent_identifier                object
gnis_id                             object
areasqkm                           float64
elevation                          float64
ATTRIBUTE                           object
WETLAND_TYPE                        object
ACRES                              float64
NWI_ID                              object
buffer                            geometry
dtype: object


In [48]:
from shapely.geometry import shape

# Convert each entry to a Shapely geometry (if not already)
aggregated_bloom_data['buffer'] = aggregated_bloom_data['buffer'].apply(lambda x: shape(x) if isinstance(x, np.ndarray) else x)


In [49]:
# Check the data type of the 'buffer' column
print(aggregated_bloom_data['buffer'].dtype)

# Check the geometry type for a sample
print([type(x) for x in aggregated_bloom_data['buffer'].head()])


geometry
[<class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>]


In [50]:
# Check for invalid geometries
invalid_geometries = aggregated_bloom_data[~aggregated_bloom_data.is_valid]
print(invalid_geometries)

# Optionally fix invalid geometries
aggregated_bloom_data['buffer'] = aggregated_bloom_data['buffer'].buffer(0)


Empty GeoDataFrame
Columns: [date_advisory_issued, lake, town, dominant_taxa, total_cell_concentration, advisory_days, notification_type, year, YearMonth, Shape_Length, Shape_Area, permanent_identifier, gnis_id, areasqkm, elevation, ATTRIBUTE, WETLAND_TYPE, ACRES, NWI_ID, buffer]
Index: []


In [64]:
# Save to GeoPackage
aggregated_bloom_data = aggregated_bloom_data.set_crs(epsg=3857)
aggregated_bloom_data.to_file('Data/aggregated_bloom_data_buffer_1km.gpkg', driver='GPKG')


In [52]:
# Inspect the entire GeoDataFrame structure
print(aggregated_bloom_data.info())

# Check the first few rows to inspect the data
print(aggregated_bloom_data.head())


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date_advisory_issued      659 non-null    datetime64[ns]
 1   lake                      659 non-null    object        
 2   town                      659 non-null    object        
 3   dominant_taxa             506 non-null    object        
 4   total_cell_concentration  419 non-null    object        
 5   advisory_days             486 non-null    float64       
 6   notification_type         659 non-null    object        
 7   year                      659 non-null    int64         
 8   YearMonth                 659 non-null    object        
 9   Shape_Length              617 non-null    float64       
 10  Shape_Area                617 non-null    float64       
 11  permanent_identifier      617 non-null    object        
 12  gnis_id       

In [53]:
# Check the type of each entry in the 'buffer' column
types_in_buffer = [type(x) for x in aggregated_bloom_data['buffer']]
print(types_in_buffer[:20])  # Print the first 20 types to inspect


[<class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.multipolygon.MultiPolygon'>, <class 'shapely.geometry.multipolygon.MultiPolygon'>, <class 'shapely.geometry.multipolygon.MultiPolygon'>, <class 'shapely.geometry.multipolygon.MultiPolygon'>, <class 'shapely.geometry.multipolygon.MultiPolygon'>, <class 'shapely.geometry.polygon.Polygon'>, <class 'shapely.geometry.polygon.Polygon'>]


In [54]:
# Check the type of the first entry in each column
for column in aggregated_bloom_data.columns:
    column_types = [type(x) for x in aggregated_bloom_data[column]]
    print(f"Column: {column}")
    print(column_types[:5])  # Show the type of the first 5 entries in each column
    print()


Column: date_advisory_issued
[<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>]

Column: lake
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]

Column: town
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]

Column: dominant_taxa
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'NoneType'>, <class 'str'>]

Column: total_cell_concentration
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'NoneType'>, <class 'str'>]

Column: advisory_days
[<class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>]

Column: notification_type
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]

Column: year
[<class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>]

Column: 

In [55]:
aggregated_bloom_data['ATTRIBUTE']

0      [L1ABH, L1UBH]
1      [L1ABH, L1UBH]
2      [L1UBH, L1ABH]
3      [L1UBH, L1ABH]
4      [L1UBH, L1ABH]
            ...      
654           [L1UBH]
655           [L1UBH]
656          [L2UBHh]
657          [L2UBHh]
658          [L2UBHh]
Name: ATTRIBUTE, Length: 659, dtype: object

In [57]:
# Assuming the two classification columns are 'classification_1' and 'classification_2'
aggregated_bloom_data['ATTRIBUTE'] = aggregated_bloom_data['ATTRIBUTE'].apply(lambda x: ','.join(x) if isinstance(x, np.ndarray) else x)
aggregated_bloom_data['WETLAND_TYPE'] = aggregated_bloom_data['WETLAND_TYPE'].apply(lambda x: ','.join(x) if isinstance(x, np.ndarray) else x)


In [63]:
aggregated_bloom_data

,date_advisory_issued,lake,town,dominant_taxa,total_cell_concentration,advisory_days,notification_type,year,YearMonth,Shape_Length,Shape_Area,permanent_identifier,gnis_id,areasqkm,elevation,ATTRIBUTE,WETLAND_TYPE,ACRES,NWI_ID,buffer
0,2019-10-11,Adder Pond/Hopkins Pond,Andover,Anabaena/Dolichospermum,"3,580,000",41.0,Warning,2019,2019-10,0.015388,0.000012,135993959,00865203,0.109221,196.3,"L1ABH,L1UBH",Lake,28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...,"POLYGON ((-7994994.165 5380215.739, -7994994.1..."
1,2022-10-06,Adder Pond/Hopkins Pond,Andover,Chrysosporum,"3,100,000",43.0,Warning,2022,2022-10,0.015388,0.000012,135993959,00865203,0.109221,196.3,"L1ABH,L1UBH",Lake,28.395538,202409CSw{1ADFAE7F-219D-4F46-B4A0-77028F0E126F...,"POLYGON ((-7994994.165 5380215.739, -7994994.1..."
2,2010-05-25,Angle Pond,Sandown,Anabaena,"235,000",9.0,Warning,2010,2010-05,0.049024,0.000072,135909961,00865245,0.650459,67.1,"L1UBH,L1ABH",Lake,161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
3,2021-08-25,Angle Pond,Sandown,None,None,NaN,Alert,2021,2021-08,0.049024,0.000072,135909961,00865245,0.650459,67.1,"L1UBH,L1ABH",Lake,161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
4,2022-09-12,Angle Pond,Sandown,Dolichospermum,"138,200",7.0,Warning,2022,2022-09,0.049024,0.000072,135909961,00865245,0.650459,67.1,"L1UBH,L1ABH",Lake,161.385552,202409CSw{37230243-4B9F-4054-B052-039EC1BA33BB...,"POLYGON ((-7923107.436 5297419.249, -7923104.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2020-08-09,"Winona, Lake",New Hampton,None,None,NaN,Alert,2020,2020-08,NaN,NaN,None,None,NaN,NaN,L1UBH,Lake,109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...,"POLYGON ((-7966193.193 5415929.044, -7966193.1..."
655,2023-07-27,"Winona, Lake",New Hampton,Dolichospermum,"950,000",7.0,Warning,2023,2023-07,NaN,NaN,None,None,NaN,NaN,L1UBH,Lake,109.462028,202409CSw{E8D14D59-7A54-4039-A53E-D19B726068EC...,"POLYGON ((-7966193.193 5415929.044, -7966193.1..."
656,2005-08-03,York Pond,Berlin,Aphanizomenon,">70,000 or >50%",48.0,Warning,2005,2005-08,0.016025,0.000010,91154789,00871018,0.087565,455.1,L2UBHh,Lake,23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...,"POLYGON ((-7942431.517 5543743.130, -7942426.0..."
657,2008-07-30,York Pond,Berlin,Aphanizomenon,">70,000 or >50%",124.0,Warning,2008,2008-07,0.016025,0.000010,91154789,00871018,0.087565,455.1,L2UBHh,Lake,23.055085,202409CSw{C1C8CD5D-99E9-48D3-A9BA-33211910441A...,"POLYGON ((-7942431.517 5543743.130, -7942426.0..."
